In [1]:
import pandas as pd
import pickle

In [2]:
#https://jsfiddle.net/gh/get/library/pure/highcharts/highcharts/tree/master/samples/highcharts/series-networkgraph/data-options/

In [1]:
#df = pd.read_csv('/Users/eric/Downloads/fr-esr-focus-collab_h2020.csv', sep=';')
#df.head()

In [5]:
#df = pd.read_csv('/Users/eric/Downloads/participants_links.csv', sep=';')
#df.head()

In [6]:
nodes = {}
projects = {}

for i, row in df.iterrows():
    inst_id = row.id_part
    if inst_id != inst_id:
        continue
    if inst_id not in nodes:
        label = row.acronym_part
        if not isinstance(label, str):
            label = row.name_part
        nodes[inst_id] = {'country': row.country_name_part, 'name': row.name_part, 
                          'label': label,
                          'country_level_part': row.country_level_part,
                          'acronym': row.acronym_part, 'all':{'all':[]}}
    
    pilier = row.pilier
    programme = f"{row.programme_lib} ({row.programme_abbr})"
    
    if pilier not in nodes[inst_id]:
        nodes[inst_id][pilier] = {}
    if programme not in nodes[inst_id][pilier]:
        nodes[inst_id][pilier][programme] = []
    if row.project_id not in nodes[inst_id][pilier][programme]:
        nodes[inst_id][pilier][programme].append(row.project_id)
    if row.project_id not in nodes[inst_id]['all']['all']:
        nodes[inst_id]['all']['all'].append(row.project_id)
    
    
    if row.project_id not in projects:
        projects[row.project_id] = {'participants': [], 'pilier': row.pilier,
                                   'programme_abbr': row.programme_abbr, 
                                   'programme_lib': row.programme_lib}
    if inst_id not in projects[row.project_id]['participants']:
        projects[row.project_id]['participants'].append(inst_id)    
        
print(len(nodes))
print(len(projects))

pickle.dump(nodes, open('nodes.pkl', 'wb'))
pickle.dump(projects, open('projects.pkl', 'wb'))

9554
5581


In [7]:
nodes['794484808']

{'country': 'France',
 'name': 'Nanoxplore',
 'label': 'Nanoxplore',
 'country_level_part': 'France',
 'acronym': 'Nanoxplore',
 'all': {'all': [101004203,
   870415,
   687220,
   730011,
   101004206,
   821969,
   870356]},
 'Industrial Leadership': {'Space (SPACE)': [101004203,
   870415,
   687220,
   730011,
   101004206,
   821969,
   870356]}}

In [10]:
df.columns

Index(['project_id', 'pilier', 'programme_abbr', 'programme_lib', 'action2_id',
       'action2_lib', 'id_gref', 'name', 'acronym', 'id_part', 'name_part',
       'acronym_part', 'country_code_part', 'country_name_part',
       'country_level_part'],
      dtype='object')

# Load

In [11]:
nodes = pickle.load(open('nodes.pkl', 'rb'))
print(len(nodes))
projects = pickle.load(open('projects.pkl', 'rb'))
print(len(projects))

9554
5581


In [12]:
list(nodes.keys())[3]

'R03cv38k47'

In [13]:
nodes['379716764']

{'country': 'France',
 'name': 'Energy Cities / Energie-Cites',
 'label': 'Energy Cities',
 'country_level_part': 'France',
 'acronym': 'Energy Cities',
 'all': {'all': [101022965,
   847136,
   894478,
   892560,
   691876,
   864212,
   869429,
   731264,
   754112,
   101036519,
   957819,
   101036640,
   101033676,
   695923]},
 'Societal Challenges': {'Secure, clean and efficient energy (ENERGY)': [101022965,
   847136,
   894478,
   892560,
   691876,
   864212,
   731264,
   754112,
   957819,
   101033676,
   695923],
  'Climate action, environment, resource efficiency and raw materials (ENV)': [869429,
   101036519],
  'Europe in a changing world - inclusive, innovative and reflective Societies (SOCIETY)': [101036640]}}

In [14]:
from itertools import combinations

def prepare_data(MY_ID):
    links = []
    graph_nodes = []

    data = {}
    
    
    pairs = {}
    THRESHOLD = 1
    for p in projects:
        projects[p]['participants'] = list(set(projects[p]['participants']))
        projects[p]['nb_participants'] = len(projects[p]['participants'])
        for i in combinations(projects[p]['participants'],2):
            # if nodes[i[0]]['country'] == "France" or nodes[i[1]]['country'] == "France":
            if i[0] == MY_ID or i[1] == MY_ID:

                if i[0] < i[1]:
                    pair = f'{i[0]};{i[1]}'
                else:
                    pair = f'{i[1]};{i[0]}'
                if pair not in pairs:
                    pairs[pair] = {}
                pilier = projects[p]['pilier']
                programme = f"{projects[p]['programme_lib']} ({projects[p]['programme_abbr']})"
                if 'all' not in pairs[pair]:
                    pairs[pair]['all'] = {}
                    pairs[pair]['all']['all'] = []
                if pilier not in pairs[pair]:
                    pairs[pair][pilier] = {}
                if programme not in pairs[pair][pilier]:
                    pairs[pair][pilier][programme] = []
                pairs[pair][pilier][programme].append(p)
                if p not in pairs[pair]['all']['all']:
                    pairs[pair]['all']['all'].append(p)
                    
    for pair in pairs:            
        id1 = pair.split(';')[0]
        id2 = pair.split(';')[1]
        for current_id in [id1, id2]:
            if current_id == MY_ID:
                continue
                
            for pilier in pairs[pair]:
                if pilier not in data:
                    data[pilier] = {}
                for programme in pairs[pair][pilier]:
                    if programme not in data[pilier]:
                        data[pilier][programme] = []
                    elt = {
                    'name': f"{nodes[current_id]['label']} ({nodes[current_id]['country']})",
                    'country': nodes[current_id]['country'],
                    'full_name': nodes[current_id]['name'],
                    'acronym': nodes[current_id]['acronym'],
                    'id': current_id,
                    'country_level_part': nodes[current_id]['country_level_part'],
                    'projects': pairs[pair][pilier][programme],
                    'y': len(pairs[pair][pilier][programme]),
                    'z': len(nodes[current_id][pilier][programme])
                    }
                    fields  = list(elt.keys()).copy()
                    for f in fields:
                        if elt[f] != elt[f]:
                            del elt[f]
                    data[pilier][programme].append(elt)

            
    for pilier in data:
        for programme in data[pilier]:
            data[pilier][programme] = sorted(data[pilier][programme], key=lambda x: x['y'], reverse=True)
    #new_data = list(data.values())
    #return sorted(new_data, key=lambda x: x['y'], reverse=True)[0:50]
    return data

In [15]:
x = prepare_data('379716764')
x

{'all': {'all': [{'name': 'Fraunhofer Society (Allemagne)',
    'country': 'Allemagne',
    'full_name': 'Fraunhofer Society',
    'id': 'R05hkkdn48',
    'country_level_part': 'Pays membres ou associés',
    'projects': [691876, 101036640, 892560, 101036519],
    'y': 4,
    'z': 626},
   {'name': 'Austrian Institute of Technology (Autriche)',
    'country': 'Autriche',
    'full_name': 'Austrian Institute of Technology',
    'id': 'R04knbh022',
    'country_level_part': 'Pays membres ou associés',
    'projects': [691876, 957819, 101036519],
    'y': 3,
    'z': 117},
   {'name': 'Hespul (France)',
    'country': 'France',
    'full_name': 'Hespul',
    'acronym': 'Hespul',
    'id': '402178701',
    'country_level_part': 'France',
    'projects': [691876, 892560],
    'y': 2,
    'z': 2},
   {'name': 'Anglia Ruskin University (Royaume-Uni)',
    'country': 'Royaume-Uni',
    'full_name': 'Anglia Ruskin University',
    'id': 'R0009t4v78',
    'country_level_part': 'Pays membres ou a

In [16]:
x['all']['all']

[{'name': 'Fraunhofer Society (Allemagne)',
  'country': 'Allemagne',
  'full_name': 'Fraunhofer Society',
  'id': 'R05hkkdn48',
  'country_level_part': 'Pays membres ou associés',
  'projects': [691876, 101036640, 892560, 101036519],
  'y': 4,
  'z': 626},
 {'name': 'Austrian Institute of Technology (Autriche)',
  'country': 'Autriche',
  'full_name': 'Austrian Institute of Technology',
  'id': 'R04knbh022',
  'country_level_part': 'Pays membres ou associés',
  'projects': [691876, 957819, 101036519],
  'y': 3,
  'z': 117},
 {'name': 'Hespul (France)',
  'country': 'France',
  'full_name': 'Hespul',
  'acronym': 'Hespul',
  'id': '402178701',
  'country_level_part': 'France',
  'projects': [691876, 892560],
  'y': 2,
  'z': 2},
 {'name': 'Anglia Ruskin University (Royaume-Uni)',
  'country': 'Royaume-Uni',
  'full_name': 'Anglia Ruskin University',
  'id': 'R0009t4v78',
  'country_level_part': 'Pays membres ou associés',
  'projects': [731264, 101036640],
  'y': 2,
  'z': 5},
 {'name'

In [17]:
import json
links_data = {}
ix = 0
nodes_fr = []
for n in nodes:
    if nodes[n]['country'] != 'France':
        continue
    #json.dump(prepare_data(n), open(f'/Users/eric/MESRI/scanr/src/Components/Focus/Data/h2020/{n}.json', 'w'), ensure_ascii=False, indent=2)

    ix += 1
    new_elt = {}
    new_elt['id'] = n
    new_elt['nb_projects'] = len(nodes[n]['all']['all'])
    new_elt['full_name'] = f"{nodes[n]['name']} ({nodes[n]['country']})"
    #nodes[n]['nb_projects'] = len(nodes[n]['projects'])
    #del nodes[n]['projects']
    nodes_fr.append(new_elt)

nodes_fr = sorted(nodes_fr, key=lambda x: x['nb_projects'], reverse=True)


In [18]:
import os
os.system("rm -rf /Users/eric/MESRI/scanr/notebooks/static/data/h2020")
os.system("mkdir -p /Users/eric/MESRI/scanr/notebooks/static/data/h2020")

0

In [19]:
json.dump(nodes_fr, open('/Users/eric/MESRI/scanr/notebooks/static/data/h2020/nodes_fr.json', 'w'), ensure_ascii=False, indent=2)

In [21]:
for node in nodes_fr:
    node_id = node['id']
    #print(node_id)
    x = prepare_data(node_id)
    json.dump(x, open(f'/Users/eric/MESRI/scanr/notebooks/static/data/h2020/{node_id}.json', 'w'), ensure_ascii=False, indent=2)


In [22]:
init_cmd = ""

os.system(init_cmd + " upload scanR static/data/h2020/")

0